# Testing for Detection

In [1]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [103]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import joblib
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

# Download NLTK resources if not already downloaded
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')  # Ensure the punkt tokenizer models are also downloaded

# Function for cleaning text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters (keep alphanumeric and spaces)
    text = re.sub(r'\d+', '', text)  # Remove digits
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Remove stop words
    return text

# Function for tokenizing and lemmatizing text
def tokenise_and_lemmatize(text):
    tokens = nltk.word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text

# Function to load trained models
def load_models():
    # Load the trained Naive Bayes classifier
    nb_classifier = joblib.load('Models/naive_bayes_model.pkl')
    dt_classifier = joblib.load('Models/decision_tree_model.pkl')
    knn_classifier = joblib.load('Models/knn_model.pkl')
    # Load the TF-IDF vectorizer
    vectorizer = joblib.load('Models/tfidf_vectorizer.pkl')
    # Load the BERT model
    bert_model = tf.keras.models.load_model('Models/bert_model.h5', custom_objects={'KerasLayer': hub.KerasLayer})
    # Load Ensemble Learning
    stacking_model = joblib.load('Models/stacking_model.pkl')

    return nb_classifier, dt_classifier, knn_classifier, vectorizer, bert_model, stacking_model

# Function to predict fraudulence
def predict_fraudulence(input_text):
    # Clean and preprocess input text
    cleaned_input = tokenise_and_lemmatize(clean_text(input_text))
    
    # Load Models 
    nb_classifier, dt_classifier, knn_classifier, vectorizer, bert_model, stacking_model  = load_models()
    
    # Vectorize input text using TF-IDF vectorizer
    X_input = vectorizer.transform([cleaned_input])
    
    # Initialize an empty list to store model names and predictions
    results = []
    
    # Predict using each trained model and append results to the list
    nb_pred = nb_classifier.predict(X_input)[0]  # Predict using Naive Bayes
    dt_pred = dt_classifier.predict(X_input)[0]  # Predict using Decision Tree
    knn_pred = knn_classifier.predict(X_input)[0]  # Predict using KN
    
    # Prepare input for BERT model
    text_input = [input_text]  # BERT model expects a list of strings
    # Get prediction from BERT model
    bert_prediction = bert_model.predict(text_input)
    bert_prediction_class = 1 if bert_prediction >= 0.5 else 0
    
    stacked_input = [[nb_pred, dt_pred, knn_pred, bert_prediction_class]]
    stacking_prediction = stacking_model.predict(stacked_input)[0]
    
    appendResults('NaiveBayes',nb_pred,results)
    appendResults('Decision Tree',dt_pred,results)
    appendResults('KNN',knn_pred,results)
    appendResults('BERT',bert_prediction_class,results)
    appendResults('Ensemble Learning',stacking_prediction,results)
    
    # Return results list containing model names and predictions
    return results

def appendResults(model_name, prediction, results_list):
    results_list.append({
        'Model':model_name,
        'Prediction':prediction
    })

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Timot\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Timot\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Timot\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Testing for Real Jobs

In [104]:
input_text = '''account executive sydney sale adthena uk leading competitive intelligence service google search advertiser adthena loved major brand digital agency alike provides great opportunity work high growth adtech space patentpending technology provide unparalleled accuracy client understand competitor keywords budget spend cpcs adcopy profitable fast growing love interested satisfying financially rewarding role high growth technology company youll work casual yet high energy environment alongside passionate people delivering leading competitive intelligence solution major global brand continued rapid growth digital marketing ppc huge opportunity exists expand adthena enterprise client base seeking experienced account executive develop close new business enterprise account large opportunity exists across major search engine vertical including auto retail gaming travel finance insurance educationprimarily youll use consultative approach determine customer need deliver presentation technical demonstration required work client advertising agency uncover value prospective customer youll supported marketing business development rep build pipeline account youll need smart passionate year experience selling softwaresaas ideally including familiarity ppc marketing technology excellent presentation communication skill well understanding marketing technology enterprise organisation entrepreneurial selfstarter looking high growth technology environment strong skill url_dafbedafddffdfbcda powerpoint google apps return well pay well give ownership company stock option importantly provide excellent opportunity advancement professional development oh well give new pair adidas trainer join associate bachelor degree'''
results = predict_fraudulence(input_text)
checkandprintresults(results)

1/1 [==============================] - 1s 751ms/step
NaiveBayes predicted that this job posting is real 
Decision Tree predicted that this job posting is real 
KNN predicted that this job posting is real 
BERT predicted that this job posting is real 
Ensemble Learning predicted that this job posting is real 


In [ ]:
input_text = '''administrativefront desk novitex enterprise solution formerly pitney bowes management service delivers innovative document communication management solution help company around world drive business process efficiency increase productivity reduce cost improve customer satisfaction almost year client turned u integrate optimize enterprisewide business process empower employee increase productivity maximize result trusted partner continually focus delivering secure technologyenabled document communication solution improve client work process enhance customer interaction drive growth customer service technical specialist based tulsa ok client location right candidate integral part talented team supporting continued growth responsibilitiessimple typingdata entry work ticket reportscovering phone help customer front desksupporting helping copy service customer service countercontacting equipment provider service necessaryordering supply keeping track inventorymaintaining log reporting documentation attention detailhandling timesensitive material like confidential urgent projectsmaintaining highest level customer care demonstrating friendly cooperative attitudedemonstrating flexibility satisfying customer demand high volume production environmentconsistently adhering business procedure guidelinestaking direction supervisor site managerperforming task assigned participate crosstrainingadhering safety procedure required qualificationsminimum year customer service related experience requiredminimum year administrative work within business production environmentstrong working knowledge pc windowsstrong working knowledge m officeability interface end user professional manner sense urgencymust able work well customer able handle fastpaced environmentexcellent communication skill verbal telephone faceface writtenability effectively work individually team environmentmust attention awareness detail specification print projectscompetency performing multiple functional tasksability meet employer attendance policylifting andor moving item pound maximum allowed current state law without accommodationsstanding andor walking long period time without accommodationshighschool diploma equivalent requiredpreferred qualificationsexperience copy print production preferredminimum year related experience preferred administrative workknowledge hardwaresoftware plus adobe photoshop indesign m officemid level access experience plusgraphic art andor prepress would pluscollege degree equivalent preferred entry level high school equivalent'''
results = predict_fraudulence(input_text)
checkandprintresults(results)

In [ ]:
input_text = '''technical project manager gbi growing company developing several cutting edge system financial industry set embark exciting new integration new partner seeking individual lead execute new project design implementation support selfmotivated detailoriented energetic passion leading facet software development project building team bring bear company descriptionhighly motivated individual quickly come speed understand support existing applicationsexperience business analysis requirement gathering functional specification testing support new existing applicationsdemonstrated ability working user goto person application related issue ability quickly become subject matter expertutilize leadership organizational skill effectively manage multiple task interface partner client executive team ensure success needed taskssupport meeting various level user management helping document support followup meeting priority must excellent oral written communication skill conduct meeting communicate effectively level management needed ability document software design adhering company standard effectively delegate software implementation year experience project management business analysis testing experience supporting web application custom application year experience working agile development project utilizing source control wiki storyboards etc experience atlassian suite zen desk similar experience crm sugarcrmpast employment history financial company associate bachelor degree'''
results = predict_fraudulence(input_text)
checkandprintresults(results)

# Test for Fake Job Postings

In [ ]:
input_text = '''director strategy engineering bright edge aptitude staffing solution partnered effort streamline hiring process provide efficient effective recruitng model focus help develop achieve career goal makeing solid geographical cultural professional fiit leveraging career new exciting professional venturebrightedge today employee strong continuously growing brightedge brand direct customer lead market seo innovation new headquarters reflect brightedges growth maturity company sleek modern space heart silicon valleystay tuned brightedges story beginning director strategy san mateo ca interested company doubled annually last three year benchmarking top percentile saas company ultimately gone public deeply capitalized backed investor battery venture intel capital descriptionthis role responsible building leading newly formed corporate strategy function report senior executive responsible identifying answering host strategic question including growth international expansion acquisition functional issue extremely visible role company work across entire organization role also responsible recruiting additional talent function developing operational process designing talent managementcareer path staff due rapid growth company role opportunity transition functional line management time interested responsibilitiesevaluation industry trend develop implication recommend strategiesdevelopment strategic recommendationsevaluation new business opportunitiesnew market analysis entry strategiescreation new initiative drawing integrated offering across multiple business unitsevaluation new technology requirementsprevious experience toptier management consulting firmqualitative quantitative analyticalfinancial analysis ability good judgment excellent problem solving skillsexcellent project management ability prioritize among competing prioritiesa history successfully engaging internal external stakeholder various level across function transformational new idea adept moving senior management actionexcellent written verbal communication skill including confidence polish routinely interact executive level audiencethe ideal candidate experience management consulting firm internal corporate strategy developing profitable growth strategiesprevious experience software required though helpfulfunctional experience strategic planning consulting finance venture capital software operationsba required bachelor degree'''
results = predict_fraudulence(input_text)
checkandprintresults(results)

In [ ]:
input_text = '''Technician Instrument & Controls  Power Plant & Energy Edison International and Refined Resources have partnered up in an effort to streamline the hiring process and provide a more efficient and effective recruitng model.Â  Our focus is to help develop and achieve your career goals while makeing a solid geographical, cultural and professional fiit when leveraging your career into your new and exciting professional venture!Please direct all communications to the HR Department within Refined Resources (#URL_80d75e0d07ca8b108539318a0443bfe5d1ff472afa0c4540b77079c5d5f31eee#)Â #EMAIL_0b13a2cfd4718ce252c09b2353d692a73bd32552e922c5db6cad5fb7e9a2c6c3#Darren Lawson | VP of Recruiting |Â #EMAIL_395225df8eed70288fc67310349d63d49d5f2ca6bc14dbb5dcbf9296069ad88c#Â | #PHONE_70128aad0c118273b0c2198a08d528591b932924e165b6a8d1272a6f9e2763d1# Technician Instrument &amp; ControlsLocation Deweyville, TXLocation Name: NRG Cottonwood EnergyCategory: MaintenanceState/City: US-TX-DeweyvilleType: Full Time Hourly# Openings: 1Details About this Career Opportunity:Overview:Company Profile:We deliver cleaner and smarter energy choices for our customers, backed by the nationâ€™s largest independent power generation portfolio of fossil fuel, nuclear, solar and wind facilities. A Fortune 500 company is challenging the U.S. energy industry by becoming the largest developer of solar power, building the first privately-funded electric vehicle charging infrastructure, and providing customers with the most advanced smart energy solutions to better manage their energy use. In addition to 47,000 megawatts of generation capacity, enough to supply nearly 40 million homes, our retail electricity providers serve more than two million customers.GENERAL SUMMARYThe Instrumentation Controls and Electrical Technician (IC&amp;E) is responsible for the maintenance and repair of all instruments, control systems and electrical equipment in the power plant and the associated facilities. The IC&amp;E Technician performs work of high skill in the inspection repair, testing, adjustment, installation and removal of all electronic and electrical power plant equipment and systems.PRIMARY FUNCTIONS-Safely perform preventive and corrective maintenance on all plant equipment, systems, and emergency backup systems.-Maintain complete equipment files, logs, and drawings for all designated equipment.-Assist in maintaining and controlling spare parts inventory to maintain plant reliability. Assist in physical inventories.-Participate in the administrative maintenance of the plant operating manuals, technical references, prints and diagrams and preventive/predictive maintenance records.-Inspect repair, install and remove all types of motors, generators, transformers, switches, control equipment, wiring, electrical signal and communication systems, and storage batteries, and work with electrical systems up to and including 4160 volts.-Conduct trouble shooting and analysis on DCS, PLC and GE Mark VI turbine control equipment.-Maintain all plant instrumentation and electrical test equipment.-Fully support and participate in NAES Safety and Regulatory Compliance Programs.-Complete other tasks as assigned by the Maintenance Manager or the Plant Manager.SCOPE OF RESPONSIBILITYThe IC&amp;E Technician operates from established and well-known procedures under minimum supervision, performing duties independently with only general direction given. Decisions are made within prescribed operating and casualty procedures and guidelines. The ICE Technician encounters recurring work situations of high complexity, with occasional variations from the norm.The IC&amp;E Technician must perform work in a safe manner resulting in no accidents, in compliance with all environmental permits at all times. The ICE Technician must effectively communicate information to supervision and fellow plant personnel, relative to the condition of the plant equipment and performance, and provide suggestions for plant improvements. The ICE Technician must perform duties in such a way to result in an optimum cost-benefit ratio.The IC&amp;E Technician may also be assigned the additional responsibility of Maintenance Coordinator/Planner for the site. This is a collateral assignment given to a selected maintenance department technician to assist the Operations and Maintenance Manager with the routine planning and coordination of the site equipment maintenance.The duties include:-The planning and review of the preventative, predictive and pro-active type maintenance.-A more in-depth knowledge and ability to use the CMMS system to provide the necessary review and planning to support the O&amp;M Manager in weekly work routines.-Ability to provide the cognitive coordinated efforts necessary to ensure that weekly routines and preventative maintenance are scheduled and documented as complete when the respective maintenance technician indicates that the work has been performed.-Provide the coordination efforts necessary in support of allowing the O&amp;M Manager to provide the necessary supervision and direction of the maintenance technicians.All employees have the responsibility to both the customer and their co-workers to do the job right the first time and to ensure the customers' needs are being met. JOB QUALIFICATIONS-Ability to understand procedures and instructions specific to the area of assignment as would be acquired in four years of high school.-Graduation from an accredited Instrument Technician Program and minimum two yearsâ€™ work experience as a technician in a related industry desirable.-Four years work experience as a technician with increasing responsibility in a related industry may be substituted for the above requirement.-Experience in working with voltages up to 4160 volts highly desirable.-Must have excellent oral and written communication skills. Ability to work well with others and be self-directed, as well as apply creative solutions to unique I&amp;E problems is desirable.-Must be able to work from blueprints.-Must be skilled in the use of hand and power driven tools.-Must be proficient in computers and computer software such as MS Office, Excel, and Word; Networking, hardware, and current security requirements for NERC Compliance.Qualifications:Title/Salary Commensurate on ExperienceOfficial description on file with Human Resources we are a team of almost 8,000 employees who help power almost 40 million homes and have more than 2 million residential and business customers across the country. Together, we are powering progress every dayâ€”from California to Texas and New Jersey to Arizona. 
'''
results = predict_fraudulence(input_text)
checkandprintresults(results)